In [ ]:

import math


# PART 1: ALGORITHM


def calculate_euclidean_distance(x1, y1, x2, y2):

    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)


def calculate_total_distance(hub_x, hub_y, sensors):
   
    total = 0
    for sensor_x, sensor_y in sensors:
        distance = calculate_euclidean_distance(hub_x, hub_y, sensor_x, sensor_y)
        total += distance
    return total


def find_optimal_hub_location(sensors, step_size=0.01):
 
    if not sensors:
        return None, None, None
    
    # Step 1: Calculate search bounds
    x_coords = [x for x, y in sensors]
    y_coords = [y for x, y in sensors]
    
    min_x = min(x_coords) - 2  # Add buffer
    max_x = max(x_coords) + 2
    min_y = min(y_coords) - 2
    max_y = max(y_coords) + 2
    
    # Step 2: Initialize variables to track best solution
    min_distance = float('inf')  # Start with infinity
    best_hub_x = 0
    best_hub_y = 0
    
    # Step 3: Grid search - test all points in the grid
    current_x = min_x
    while current_x <= max_x:
        current_y = min_y
        while current_y <= max_y:
            # Calculate total distance for this point
            total_dist = calculate_total_distance(current_x, current_y, sensors)
            
            # Update best solution if this is better
            if total_dist < min_distance:
                min_distance = total_dist
                best_hub_x = current_x
                best_hub_y = current_y
            
            current_y += step_size
        current_x += step_size
    
    # Step 4: Return the optimal solution
    return round(best_hub_x, 5), round(best_hub_y, 5), round(min_distance, 5)


def solve_sensor_placement(sensor_locations):
   
    # Convert to list of tuples for easier processing
    sensors = [(x, y) for x, y in sensor_locations]
    
    # Find optimal hub
    hub_x, hub_y, min_distance = find_optimal_hub_location(sensors)
    
    # Calculate individual distances
    individual_distances = []
    if hub_x is not None:
        for i, (sx, sy) in enumerate(sensors):
            dist = calculate_euclidean_distance(hub_x, hub_y, sx, sy)
            individual_distances.append({
                'sensor_id': i + 1,
                'distance': round(dist, 5)
            })
    
    return {
        'hub_location': (hub_x, hub_y),
        'minimum_total_distance': min_distance,
        'individual_distances': individual_distances
    }




# TESTING THE ALGORITHM


def test_examples():
  
    
    print("="*70)
    print("TESTING OPTIMAL SENSOR PLACEMENT ALGORITHM")
    print("="*70)
    
    # Example 1
    print("\nExample 1:")
    sensors1 = [[0, 1], [1, 0], [1, 2], [2, 1]]
    result1 = solve_sensor_placement(sensors1)
    print(f"Input: {sensors1}")
    print(f"Optimal Hub: {result1['hub_location']}")
    print(f"Minimum Distance: {result1['minimum_total_distance']}")
    print(f"Expected: 4.00000")
    
    # Example 2
    print("\nExample 2:")
    sensors2 = [[1, 1], [3, 3]]
    result2 = solve_sensor_placement(sensors2)
    print(f"Input: {sensors2}")
    print(f"Optimal Hub: {result2['hub_location']}")
    print(f"Minimum Distance: {result2['minimum_total_distance']}")
    print(f"Expected: 2.82843")
    
    print("\n" + "="*70)



# PART 2: GUI IMPLEMENTATION USING

import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure


class SensorPlacementGUI:
    
    
    def __init__(self, root):
        """Initialize the GUI application."""
        self.root = root
        self.root.title("Optimal Sensor Placement Calculator")
        self.root.geometry("1200x750")
        self.root.configure(bg="#f0f4f8")
        
        # Data storage
        self.sensor_entries = [] 
        self.optimal_hub = None
        self.min_distance = None
        
        # Build the GUI
        self.setup_gui()
        
        # Load example 1 by default
        self.load_example_1()
    
    def setup_gui(self):
        """Create all GUI components."""
        # Main container
        main_container = tk.Frame(self.root, bg="#f0f4f8")
        main_container.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Title section
        self.create_title_section(main_container)
        
        # Content area with two panels
        content_frame = tk.Frame(main_container, bg="#f0f4f8")
        content_frame.pack(fill=tk.BOTH, expand=True)
        
        # Left panel: Input controls
        self.create_input_panel(content_frame)
        
        # Right panel: Visualization
        self.create_visualization_panel(content_frame)
    
    def create_title_section(self, parent):
        """Create the title and subtitle."""
        title_frame = tk.Frame(parent, bg="#f0f4f8")
        title_frame.pack(fill=tk.X, pady=(0, 15))
        
        tk.Label(
            title_frame,
            text="Optimal Sensor Placement Calculator",
            font=("Arial", 22, "bold"),
            bg="#f0f4f8",
            fg="#1e40af"
        ).pack()
        
        tk.Label(
            title_frame,
            text="Find the optimal data aggregation hub using the Geometric Median algorithm",
            font=("Arial", 11),
            bg="#f0f4f8",
            fg="#64748b"
        ).pack()
    
    def create_input_panel(self, parent):
        # Panel container
        left_panel = tk.Frame(parent, bg="white", relief=tk.RIDGE, borderwidth=2)
        left_panel.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=(0, 5))
        
        # Header with buttons
        header = tk.Frame(left_panel, bg="white")
        header.pack(fill=tk.X, padx=10, pady=10)
        
        tk.Label(
            header,
            text="📍 Sensor Locations",
            font=("Arial", 14, "bold"),
            bg="white",
            fg="#1e40af"
        ).pack(side=tk.LEFT)
        
        btn_frame = tk.Frame(header, bg="white")
        btn_frame.pack(side=tk.RIGHT)
        
        # Add Sensor button
        tk.Button(
            btn_frame,
            text="Add",
            command=self.add_sensor,
            bg="#10b981",
            fg="white",
            font=("Arial", 10, "bold"),
            padx=12,
            pady=5,
            cursor="hand2"
        ).pack(side=tk.LEFT, padx=2)
        
        # Clear All button
        tk.Button(
            btn_frame,
            text="Clear",
            command=self.clear_all_sensors,
            bg="#ef4444",
            fg="white",
            font=("Arial", 10, "bold"),
            padx=12,
            pady=5,
            cursor="hand2"
        ).pack(side=tk.LEFT, padx=2)
        
        # Load examples
        tk.Button(
            btn_frame,
            text="📋 Ex1",
            command=self.load_example_1,
            bg="#6366f1",
            fg="white",
            font=("Arial", 10, "bold"),
            padx=8,
            pady=5,
            cursor="hand2"
        ).pack(side=tk.LEFT, padx=2)
        
        tk.Button(
            btn_frame,
            text="📋 Ex2",
            command=self.load_example_2,
            bg="#6366f1",
            fg="white",
            font=("Arial", 10, "bold"),
            padx=8,
            pady=5,
            cursor="hand2"
        ).pack(side=tk.LEFT, padx=2)
        
        # Scrollable sensor list
        list_frame = tk.Frame(left_panel, bg="white")
        list_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=(0, 10))
        
        canvas = tk.Canvas(list_frame, bg="white", highlightthickness=0)
        scrollbar = ttk.Scrollbar(list_frame, orient="vertical", command=canvas.yview)
        self.sensor_container = tk.Frame(canvas, bg="white")
        
        self.sensor_container.bind(
            "<Configure>",
            lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
        )
        
        canvas.create_window((0, 0), window=self.sensor_container, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        
        canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        # Calculate button
        tk.Button(
            left_panel,
            text="▶️ CALCULATE OPTIMAL HUB",
            command=self.calculate_hub,
            bg="#4f46e5",
            fg="white",
            font=("Arial", 13, "bold"),
            padx=20,
            pady=12,
            cursor="hand2"
        ).pack(fill=tk.X, padx=10, pady=(0, 10))
        
        # Results display
        results_frame = tk.Frame(left_panel, bg="#e0e7ff", relief=tk.RIDGE, borderwidth=2)
        results_frame.pack(fill=tk.X, padx=10, pady=(0, 10))
        
        tk.Label(
            results_frame,
            text="Results:",
            font=("Arial", 12, "bold"),
            bg="#e0e7ff",
            fg="#1e40af"
        ).pack(anchor=tk.W, padx=10, pady=(10, 5))
        
        self.results_text = scrolledtext.ScrolledText(
            results_frame,
            height=7,
            font=("Courier", 9),
            bg="#f8fafc",
            relief=tk.FLAT,
            padx=8,
            pady=8,
            wrap=tk.WORD
        )
        self.results_text.pack(fill=tk.X, padx=10, pady=(0, 10))
        self.results_text.config(state=tk.DISABLED)
    
    def create_visualization_panel(self, parent):
    
        right_panel = tk.Frame(parent, bg="white", relief=tk.RIDGE, borderwidth=2)
        right_panel.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=(5, 0))
        
        tk.Label(
            right_panel,
            text="📈 Sensor Network Visualization",
            font=("Arial", 14, "bold"),
            bg="white",
            fg="#1e40af"
        ).pack(pady=10)
        
        # Matplotlib figure
        self.fig = Figure(figsize=(6, 5.5), dpi=100, facecolor='white')
        self.ax = self.fig.add_subplot(111)
        
        self.canvas = FigureCanvasTkAgg(self.fig, right_panel)
        self.canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Information box
        info_frame = tk.Frame(right_panel, bg="#dbeafe", relief=tk.RIDGE, borderwidth=1)
        info_frame.pack(fill=tk.X, padx=10, pady=(0, 10))
        
        tk.Label(
            info_frame,
            text="💡 Algorithm Info:",
            font=("Arial", 10, "bold"),
            bg="#dbeafe",
            fg="#1e40af"
        ).pack(anchor=tk.W, padx=10, pady=(8, 3))
        
        tk.Label(
            info_frame,
            text="Uses Geometric Median (Fermat-Weber Point) with Grid Search\n"
                 "to find the optimal hub location that minimizes the sum of\n"
                 "Euclidean distances from hub to all sensors.",
            font=("Arial", 9),
            bg="#dbeafe",
            fg="#475569",
            justify=tk.LEFT
        ).pack(anchor=tk.W, padx=10, pady=(0, 8))
        
        self.draw_plot()
    
    def add_sensor(self):
        self.add_sensor_row(0, 0)
    
    def add_sensor_row(self, x_val, y_val):
        sensor_frame = tk.Frame(
            self.sensor_container,
            bg="#f1f5f9",
            relief=tk.RAISED,
            borderwidth=1
        )
        sensor_frame.pack(fill=tk.X, pady=4, padx=5)
        
        sensor_num = len(self.sensor_entries) + 1
        
        # Sensor label
        label = tk.Label(
            sensor_frame,
            text=f"Sensor {sensor_num}:",
            font=("Arial", 10, "bold"),
            bg="#f1f5f9",
            fg="#334155",
            width=10
        )
        label.pack(side=tk.LEFT, padx=5, pady=5)
        
        # X coordinate
        tk.Label(
            sensor_frame,
            text="X:",
            bg="#f1f5f9",
            fg="#64748b",
            font=("Arial", 9, "bold")
        ).pack(side=tk.LEFT, padx=(5, 2))
        
        x_entry = tk.Entry(sensor_frame, width=8, font=("Arial", 10))
        x_entry.pack(side=tk.LEFT, padx=(0, 10))
        x_entry.insert(0, str(x_val))
        
        # Y coordinate
        tk.Label(
            sensor_frame,
            text="Y:",
            bg="#f1f5f9",
            fg="#64748b",
            font=("Arial", 9, "bold")
        ).pack(side=tk.LEFT, padx=(5, 2))
        
        y_entry = tk.Entry(sensor_frame, width=8, font=("Arial", 10))
        y_entry.pack(side=tk.LEFT, padx=(0, 10))
        y_entry.insert(0, str(y_val))
        
        # Delete button
        def delete_this_sensor():
            sensor_frame.destroy()
            self.sensor_entries.remove(entry_dict)
            self.update_sensor_labels()
            self.draw_plot()
        
        tk.Button(
            sensor_frame,
            text="✖",
            command=delete_this_sensor,
            bg="#fecaca",
            fg="#991b1b",
            font=("Arial", 9, "bold"),
            relief=tk.FLAT,
            cursor="hand2",
            padx=6
        ).pack(side=tk.RIGHT, padx=5)
        
        entry_dict = {
            'frame': sensor_frame,
            'label': label,
            'x_entry': x_entry,
            'y_entry': y_entry
        }
        
        self.sensor_entries.append(entry_dict)
    
    def update_sensor_labels(self):

        for i, entry in enumerate(self.sensor_entries, 1):
            entry['label'].config(text=f"Sensor {i}:")
    
    def clear_all_sensors(self):
        if messagebox.askyesno("Confirm", "Clear all sensors?"):
            for entry in self.sensor_entries[:]:
                entry['frame'].destroy()
            self.sensor_entries.clear()
            self.optimal_hub = None
            self.min_distance = None
            self.update_results("")
            self.draw_plot()
    
    def load_example_1(self):
        self.clear_all_sensors()
        self.add_sensor_row(0, 1)
        self.add_sensor_row(1, 0)
        self.add_sensor_row(1, 2)
        self.add_sensor_row(2, 1)
        self.draw_plot()
    
    def load_example_2(self):
        """Load Example 2: [[1,1], [3,3]]"""
        self.clear_all_sensors()
        self.add_sensor_row(1, 1)
        self.add_sensor_row(3, 3)
        self.draw_plot()
    
    def get_sensor_coordinates(self):
        """Extract coordinates from all sensor entries."""
        coordinates = []
        for entry in self.sensor_entries:
            try:
                x = float(entry['x_entry'].get())
                y = float(entry['y_entry'].get())
                coordinates.append([x, y])
            except ValueError:
                messagebox.showerror("Error", "Please enter valid numbers!")
                return None
        return coordinates
    
    def calculate_hub(self):
        """Calculate optimal hub using the core algorithm."""
        sensors = self.get_sensor_coordinates()
        
        if sensors is None:
            return
        
        if len(sensors) == 0:
            messagebox.showwarning("Warning", "Please add at least one sensor!")
            return
        
        # Use the core algorithm
        result = solve_sensor_placement(sensors)
        
        self.optimal_hub = result['hub_location']
        self.min_distance = result['minimum_total_distance']
        
        # Format results
        result_text = f"""
Optimal Hub Location: {self.optimal_hub}

Minimum Total Distance: {self.min_distance}

Individual Distances:
"""
        for dist_info in result['individual_distances']:
            result_text += f"  Sensor {dist_info['sensor_id']}: {dist_info['distance']}\n"
        
        result_text += f"\nTotal Sum: {self.min_distance}"
        
        self.update_results(result_text)
        self.draw_plot()
        
        messagebox.showinfo(
            f"Optimal Hub: {self.optimal_hub}\n"
            f"Min Distance: {self.min_distance}"
        )
    
    def update_results(self, text):
        """Update the results text box."""
        self.results_text.config(state=tk.NORMAL)
        self.results_text.delete(1.0, tk.END)
        self.results_text.insert(1.0, text)
        self.results_text.config(state=tk.DISABLED)
    
    def draw_plot(self):
        """Draw the sensor network visualization."""
        self.ax.clear()
        
        sensors = self.get_sensor_coordinates()
        if sensors is None:
            sensors = []
        
        # Configure plot
        self.ax.set_xlabel('X Coordinate', fontsize=10, fontweight='bold')
        self.ax.set_ylabel('Y Coordinate', fontsize=10, fontweight='bold')
        self.ax.set_title('Sensor Network Map', fontsize=12, fontweight='bold', pad=15)
        self.ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
        self.ax.set_aspect('equal', adjustable='box')
        
        if sensors:
            # Extract coordinates
            x_coords = [s[0] for s in sensors]
            y_coords = [s[1] for s in sensors]
            
            # Set plot limits
            padding = 1
            x_min, x_max = min(x_coords) - padding, max(x_coords) + padding
            y_min, y_max = min(y_coords) - padding, max(y_coords) + padding
            
            self.ax.set_xlim(x_min, x_max)
            self.ax.set_ylim(y_min, y_max)
            
            # Draw connection lines
            if self.optimal_hub and self.optimal_hub[0] is not None:
                for x, y in sensors:
                    self.ax.plot(
                        [self.optimal_hub[0], x],
                        [self.optimal_hub[1], y],
                        'b--',
                        alpha=0.4,
                        linewidth=1.5
                    )
            
            # Draw sensors
            for i, (x, y) in enumerate(sensors, 1):
                self.ax.plot(x, y, 'bo', markersize=13, 
                           markeredgecolor='darkblue', markeredgewidth=2.5)
                self.ax.text(x + 0.15, y + 0.15, f'S{i}', 
                           fontsize=10, fontweight='bold', color='darkblue')
            
            # Draw hub
            if self.optimal_hub and self.optimal_hub[0] is not None:
                self.ax.plot(
                    self.optimal_hub[0], self.optimal_hub[1],
                    marker='*', color='gold', markersize=22,
                    markeredgecolor='orange', markeredgewidth=2.5,
                    label='Optimal Hub'
                )
                self.ax.text(
                    self.optimal_hub[0] + 0.15, self.optimal_hub[1] + 0.15,
                    'HUB', fontsize=10, fontweight='bold', color='orange'
                )
            
            # Legend
            self.ax.legend(loc='upper right', fontsize=9, framealpha=0.9)
        else:
            self.ax.text(0.5, 0.5, 'No sensors yet\nAdd sensors to begin',
                        ha='center', va='center', transform=self.ax.transAxes,
                        fontsize=13, color='gray', style='italic')
            self.ax.set_xlim(-1, 1)
            self.ax.set_ylim(-1, 1)
        
        self.canvas.draw()



def main():
    root = tk.Tk()
    app = SensorPlacementGUI(root)
    root.mainloop()


if __name__ == "__main__":
    main()
  

C:\Users\heema\AppData\Local\Temp\ipykernel_20984\1403183249.py:590: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  self.ax.legend(loc='upper right', fontsize=9, framealpha=0.9)
